In this notebook, I will investigate the het calls on the male X, and remove them based on the state present in females from the same species.
I will adapt some of the code from Primatediversity_shared_het

In [1]:
%run ../scripts/notebook_settings.py

In [2]:
meta_data_samples = pd.read_table("../data/metadata_with_x_missing.txt", sep=" ")
zarr_dir = "/faststorage/project/baboondiversity/data/PG_panu3_zarr_12_03_2021/callset.zarr/chrX"
#Opening the zarr data
callset = zarr.open_group(zarr_dir, mode="r")
gt = allel.GenotypeArray(callset["calldata/GT"])

In [125]:
meta_data_samples.loc[meta_data_samples.Species == "ursinus (grayfoot)"]

,PGDP_ID,Provider_ID,Provider,Genus,Species,Origin,Sex,address,longitude,latitude,callset_index,C_origin,x_missing
124,PD_0692,35221_BZ12030,Rogers/Jolly/Phillips-Conroy,Papio,ursinus (grayfoot),"Dendro Park, Zambia",M,"Dendro Balla Balla Safaris, Namwala, Zambia",26.06255,-16.145438,124,"Ursinus, Zambia",0.014076
125,PD_0693,35225_BZ12031,Rogers/Jolly/Phillips-Conroy,Papio,ursinus (grayfoot),"Dendro Park, Zambia",F,"Dendro Balla Balla Safaris, Namwala, Zambia",26.06255,-16.145438,125,"Ursinus, Zambia",0.009636
126,PD_0694,35227_BZ12032,Rogers/Jolly/Phillips-Conroy,Papio,ursinus (grayfoot),"Dendro Park, Zambia",F,"Dendro Balla Balla Safaris, Namwala, Zambia",26.06255,-16.145438,126,"Ursinus, Zambia",0.009243
127,PD_0695,35229_BZ12033,Rogers/Jolly/Phillips-Conroy,Papio,ursinus (grayfoot),"Dendro Park, Zambia",F,"Dendro Balla Balla Safaris, Namwala, Zambia",26.06255,-16.145438,127,"Ursinus, Zambia",0.009191


In [3]:
def count_het_species(gt, species):
    IDs = meta_data_samples.loc[meta_data_samples.Species == species].callset_index.values
    females = meta_data_samples.loc[(meta_data_samples.Species == species)
                                    & (meta_data_samples.Sex == "F")].callset_index.values
    males = meta_data_samples.loc[(meta_data_samples.Species == species)
                                    & (meta_data_samples.Sex == "M")].callset_index.values
    gt_f = gt.take(females, axis=1)
    gt_m = gt.take(males, axis=1)
    f_het = gt_f.is_het()
    m_het = gt_m.is_het()
    matrix = np.zeros((len(males)+1, len(females)+1))
    m_sum = m_het.sum(axis=1)
    f_sum = f_het.sum(axis=1)
    for i in range(len(m_sum)):
        matrix[m_sum[i], f_sum[i]] += 1
    print(species)
    print("{} males and {} females".format(len(males), len(females)))
    print("Out of {} sites, {} are present in males,".format(matrix.sum(), sum(matrix.sum(axis=1)[1:])/matrix.sum()))
    print("{} are in females".format(sum(matrix.sum(axis=0)[1:])/matrix.sum()))
    print("With {} of male sites overlapping".format(matrix[1:, 1:].sum()/sum(matrix.sum(axis=1)[1:])))
    print("{} sites are het in all".format(matrix[len(males), len(females)]/matrix.sum()))
    return matrix

In [4]:
for species in meta_data_samples.Species.unique():
    df = count_het_species(gt, species)
df

cynocephalus
38 males and 24 females
Out of 2923212.0 sites, 0.08423405486841187 are present in males,
0.3749060964445959 are in females
With 0.3710413671548202 of male sites overlapping
0.0012883088876208774 sites are het in all
anubis
42 males and 52 females
Out of 2923212.0 sites, 0.07334124244153349 are present in males,
0.29615060419839545 are in females
With 0.31978338743982987 of male sites overlapping
0.001240416363917499 sites are het in all
kindae
18 males and 9 females
Out of 2923212.0 sites, 0.06085668778042783 are present in males,
0.17061677360383032 are in females
With 0.2319488243196906 of male sites overlapping
0.0018226526163685698 sites are het in all
gelada
2 males and 0 females
Out of 2923212.0 sites, 0.046292229232775456 are present in males,
0.0 are in females
With 0.0 of male sites overlapping
0.031158191742507897 sites are het in all
hamadryas
20 males and 6 females
Out of 2923212.0 sites, 0.05886504297327734 are present in males,
0.110604020508947 are in femal

array([[2599872.,  137074.,   73899.,   16869.],
       [  79898.,    2734.,    2641.,   10225.]])

Now I remove the putative PAR.

In [5]:
callset = zarr.open_group(zarr_dir, mode="r")
gt_zarr = callset["calldata/GT"]
pos = allel.SortedIndex(callset['variants/POS'])
loc_region = pos.locate_range(2500000, 140000000)
gt_nonpar = allel.GenotypeArray(gt_zarr[loc_region])

In [6]:
gt_nonpar

<GenotypeArray shape=(2774845, 227, 2) dtype=int8>
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
1/1 1/1 0/0 0/0 1/1 ... 1/1 1/1 1/1 1/1 1/1
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

In [7]:
len(gt)

2923212

In [8]:
for species in meta_data_samples.Species.unique():
    df = count_het_species(gt_nonpar, species)

cynocephalus
38 males and 24 females
Out of 2774845.0 sites, 0.07232728314554507 are present in males,
0.38014303501637026 are in females
With 0.4039024001355277 of male sites overlapping
0.001236825840722635 sites are het in all
anubis
42 males and 52 females
Out of 2774845.0 sites, 0.062125632242521656 are present in males,
0.29948123228504653 are in females
With 0.34951185980544003 of male sites overlapping
0.0011957424648944355 sites are het in all
kindae
18 males and 9 females
Out of 2774845.0 sites, 0.05066084772302597 are present in males,
0.1738197989437248 are in females
With 0.26005861597996816 of male sites overlapping
0.0017337905360479595 sites are het in all
gelada
2 males and 0 females
Out of 2774845.0 sites, 0.038015096338714416 are present in males,
0.0 are in females
With 0.0 of male sites overlapping
0.024557407711061338 sites are het in all
hamadryas
20 males and 6 females
Out of 2774845.0 sites, 0.04971881312289515 are present in males,
0.11216806704518631 are in f

Removing PAR actually increases overlap, but it decreases the percentage of sites with male het.

In [9]:
df = count_het_species(gt_nonpar, "kindae")

kindae
18 males and 9 females
Out of 2774845.0 sites, 0.05066084772302597 are present in males,
0.1738197989437248 are in females
With 0.26005861597996816 of male sites overlapping
0.0017337905360479595 sites are het in all


Now, I will create a "reference" allele count for each female population, and then go through the males one by one to decide on the states for each site.

In [10]:
def state_decision(gt, species, meta_data_samples):
    females = meta_data_samples.loc[(meta_data_samples.Species == species)
                                    & (meta_data_samples.Sex == "F")].callset_index.values
    males = meta_data_samples.loc[(meta_data_samples.Species == species)
                                    & (meta_data_samples.Sex == "M")].callset_index.values
    gt_f = gt.take(females, axis=1)
    ac_f = gt_f.count_alleles()
    allelism_f = np.reshape(ac_f.allelism()==1, (-1,1))
    #reshape is needed to make sure they have the same np dimensions
    print("Total sites:", len(ac_f))
    print("Variants in all females: ", ac_f.is_variant().sum())
    gt_all_males = gt.take(males, axis=1)
    avg_het = gt_all_males.count_het()/len(males)
    avg_share = []
    avg_removed = []
    gt_list = []
    for ID in males:
        gt_m = gt.take([ID], axis=1)
        het_state = gt_m.is_het()
        variable_sites = gt_m.count_called().sum()-gt_m.is_hom_ref().sum()
        print("Variants in male: {}, of which {} are het".format(
            variable_sites, het_state.sum()/variable_sites))
        het_males_fixed_females = het_state & allelism_f
        print("""Fixed female: {}, Het sites in male {}, percentage {}""".format(
        het_males_fixed_females.sum(), het_state.sum(), het_males_fixed_females.sum()/het_state.sum()))
        avg_share.append(het_males_fixed_females.sum()/het_state.sum())
        mask = het_state & (allelism_f==False)
        print("Removed {} sites due to the masking".format((het_state & (allelism_f==False)).sum()))
        avg_removed.append((het_state & (allelism_f==False)).sum()/variable_sites)
        gt_m.mask = mask
        gt_m.fill_masked(value=-1, copy=False)
        max_alt = ac_f.max_allele()
        for i in range(max_alt.max()):
            alt_bool = np.reshape((max_alt==i), (-1,1)) & het_males_fixed_females
            print("{} sites in state {}".format(alt_bool.sum(), i))
            mask = alt_bool
            gt_m.mask = mask
            gt_m.fill_masked(value=i, copy=False)
        gt_list.append(gt_m)
    print("Average number of het sites in males {}, percentage which can be assigned {}. {} % removed".format(
        avg_het, sum(avg_share)/len(avg_share), sum(avg_removed)/len(avg_removed)))
    gt_merged = np.concatenate(gt_list, axis=1)
    return allel.GenotypeArray(gt_merged), males

In [50]:
gt_list = []
ID_list = []
for species in meta_data_samples.Species.unique():
    if species == "gelada":
        continue
    gts, filter_IDs = state_decision(gt_nonpar, species, meta_data_samples)
    gt_list.extend(gts)
    ID_list.extend(filter_IDs)
np_merged = np.concatenate(gt_list, axis=1)
gt_merged = allel.GenotypeArray(np_merged)

Total sites: 2774845
Variants in all females:  1148230
Variants in male: 375671, of which 0.22340558627096582 are het
Fixed female: 62572, Het sites in male 83927, percentage 0.7455526826885269
Removed 21355 sites due to the masking
62413 sites in state 0
158 sites in state 1
1 sites in state 2
0 sites in state 3
Variants in male: 377634, of which 0.2207693163221532 are het
Fixed female: 62328, Het sites in male 83370, percentage 0.7476070528967255
Removed 21042 sites due to the masking
62156 sites in state 0
171 sites in state 1
1 sites in state 2
0 sites in state 3
Variants in male: 402875, of which 0.21033571206950047 are het
Fixed female: 62065, Het sites in male 84739, percentage 0.7324254475507146
Removed 22674 sites due to the masking
61735 sites in state 0
330 sites in state 1
0 sites in state 2
0 sites in state 3
Variants in male: 401273, of which 0.21129006935428 are het
Fixed female: 62814, Het sites in male 84785, percentage 0.7408621808102849
Removed 21971 sites due to the

Variants in male: 399868, of which 0.2015039963187852 are het
Fixed female: 61966, Het sites in male 80575, percentage 0.769047471300031
Removed 18609 sites due to the masking
61839 sites in state 0
126 sites in state 1
1 sites in state 2
0 sites in state 3
Variants in male: 404223, of which 0.2009707512932218 are het
Fixed female: 61773, Het sites in male 81237, percentage 0.760404741681746
Removed 19464 sites due to the masking
61637 sites in state 0
135 sites in state 1
1 sites in state 2
0 sites in state 3
Variants in male: 397960, of which 0.201143330988039 are het
Fixed female: 60917, Het sites in male 80047, percentage 0.7610154034504728
Removed 19130 sites due to the masking
60802 sites in state 0
115 sites in state 1
0 sites in state 2
0 sites in state 3
Variants in male: 400004, of which 0.19759552404475955 are het
Fixed female: 60599, Het sites in male 79039, percentage 0.7666974531560369
Removed 18440 sites due to the masking
60467 sites in state 0
131 sites in state 1
1 si

Variants in male: 217082, of which 0.3625957011636156 are het
Fixed female: 63009, Het sites in male 78713, percentage 0.8004903891352128
Removed 15704 sites due to the masking
62977 sites in state 0
32 sites in state 1
0 sites in state 2
0 sites in state 3
Variants in male: 215323, of which 0.356701327772695 are het
Fixed female: 61188, Het sites in male 76806, percentage 0.7966565112100618
Removed 15618 sites due to the masking
61158 sites in state 0
30 sites in state 1
0 sites in state 2
0 sites in state 3
Variants in male: 210677, of which 0.338171703603146 are het
Fixed female: 56160, Het sites in male 71245, percentage 0.7882658432170678
Removed 15085 sites due to the masking
56130 sites in state 0
30 sites in state 1
0 sites in state 2
0 sites in state 3
Variants in male: 215008, of which 0.35802853847298705 are het
Fixed female: 61587, Het sites in male 76979, percentage 0.8000493641122903
Removed 15392 sites due to the masking
61548 sites in state 0
38 sites in state 1
0 sites

Variants in male: 387138, of which 0.20448263926558488 are het
Fixed female: 62912, Het sites in male 79163, percentage 0.7947147025756982
Removed 16251 sites due to the masking
62418 sites in state 0
483 sites in state 1
11 sites in state 2
0 sites in state 3
Variants in male: 376336, of which 0.1850208324476 are het
Fixed female: 54017, Het sites in male 69630, percentage 0.7757719373833117
Removed 15613 sites due to the masking
53559 sites in state 0
442 sites in state 1
15 sites in state 2
1 sites in state 3
Variants in male: 384306, of which 0.20212278757032157 are het
Fixed female: 61750, Het sites in male 77677, percentage 0.7949586106569512
Removed 15927 sites due to the masking
61248 sites in state 0
494 sites in state 1
8 sites in state 2
0 sites in state 3
Variants in male: 383930, of which 0.20389393899929675 are het
Fixed female: 62231, Het sites in male 78281, percentage 0.7949694050919125
Removed 16050 sites due to the masking
61681 sites in state 0
541 sites in state 1


Total sites: 2774845
Variants in all females:  465119
Variants in male: 385471, of which 0.1832277914551289 are het
Fixed female: 56649, Het sites in male 70629, percentage 0.8020643078622096
Removed 13980 sites due to the masking
55504 sites in state 0
1116 sites in state 1
29 sites in state 2
0 sites in state 3
Average number of het sites in males 70629.0, percentage which can be assigned 0.8020643078622096. 0.03626731972054967 % removed


In [51]:
gt_merged

<GenotypeArray shape=(2774845, 126, 2) dtype=int8>
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 1/1
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
1/1 1/1 1/1 1/1 1/1 ... 0/0 0/0 0/0 0/0 1/1
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 1/1 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

In [59]:
ht_merged = gt_merged.haploidify_samples()

In [52]:
meta_subset = meta_data_samples.loc[meta_data_samples.callset_index.isin(ID_list)]
meta_subset = meta_subset.reindex(ID_list).reset_index(drop=True)
meta_subset["callset_index"] = list(range(len(meta_subset)))
meta_subset

,PGDP_ID,Provider_ID,Provider,Genus,Species,Origin,Sex,address,longitude,latitude,callset_index,C_origin,x_missing
0,PD_0215,13RUM1130716,Knauf/Chuma/Roos,Papio,cynocephalus,"Ruaha, Tanzania",M,"Ruaha, Ulanga, Morogoro, Coastal Zone, Tanzania",36.714814,-8.895386,0,"Cynocephalus, Central Tanzania",0.014343
1,PD_0217,17RUM5140716,Knauf/Chuma/Roos,Papio,cynocephalus,"Ruaha, Tanzania",M,"Ruaha, Ulanga, Morogoro, Coastal Zone, Tanzania",36.714814,-8.895386,1,"Cynocephalus, Central Tanzania",0.015386
2,PD_0223,02UNM1220616,Knauf/Chuma/Roos,Papio,cynocephalus,"Udzungwa, Tanzania",M,"Udzungwa National Park, Iringa, Southern Highl...",36.725746,-7.805251,2,"Cynocephalus, Central Tanzania",0.015105
3,PD_0227,27UNM1160317,Knauf/Chuma/Roos,Papio,cynocephalus,"Udzungwa, Tanzania",M,"Udzungwa National Park, Iringa, Southern Highl...",36.725746,-7.805251,3,"Cynocephalus, Central Tanzania",0.014568
4,PD_0228,01MNM1300916,Knauf/Chuma/Roos,Papio,cynocephalus,"Mahale, Tanzania",M,"Mahale National Park, Kigoma, Western Zone, Ta...",29.883426,-6.203292,4,"Cynocephalus, Western Tanzania",0.015121
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,PD_0398,T37,Zinner/Fischer,Papio,papio,"Niokolo-Koba, Senegal",M,"Niokolo Koba, Kédougou, Sénégal",-12.748341,13.078081,121,"Papio, Senegal",0.012534
122,PD_0399,T49,Zinner/Fischer,Papio,papio,"Niokolo-Koba, Senegal",M,"Niokolo Koba, Kédougou, Sénégal",-12.748341,13.078081,122,"Papio, Senegal",0.012503
123,PD_0400,T51,Zinner/Fischer,Papio,papio,"Niokolo-Koba, Senegal",M,"Niokolo Koba, Kédougou, Sénégal",-12.748341,13.078081,123,"Papio, Senegal",0.011650
124,PD_0401,T66,Zinner/Fischer,Papio,papio,"Niokolo-Koba, Senegal",M,"Niokolo Koba, Kédougou, Sénégal",-12.748341,13.078081,124,"Papio, Senegal",0.012213


Checking that each gt is unique, and verify that all seems correct.

In [45]:
ID_list

[124,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 181,
 182,
 183,
 190,
 191,
 208,
 209,
 210,
 211,
 212,
 214,
 215,
 216,
 217,
 218,
 219,
 225,
 226]

In [46]:
print(gt_merged.take([1], axis=1).count_hom_alt())
c = gt_merged.take([1], axis=1).is_hom_alt() & gt_merged.take([2], axis=1).is_hom_alt()
print(c.sum())

163152
156118


In [47]:
(gt_merged.count_hom_alt()+gt_merged.count_hom_ref()+gt_merged.count_missing())/len(ID_list)

2774845.0

Setting up the columns for the vcf.
allel.write_vcf cannot handle sample data currently, so that is out of the question for creating the full vcf

In [92]:
callset.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, name='calldata', nodes=(Node(disabled=Tru…

In [111]:
chrom = callset["variants/CHROM"][loc_region]
pos = callset["variants/POS"][loc_region]
#variant_id = callset["variants/ID"][loc_region]
ref = callset["variants/REF"][loc_region]
alt = callset["variants/ALT"][loc_region]
qual = callset["variants/QUAL"][loc_region]
filter_pass = callset["variants/FILTER_PASS"][loc_region]

In [123]:
alt

array([['A', '', ''],
       ['G', '', ''],
       ['T', '', ''],
       ...,
       ['A', '', ''],
       ['A', '', ''],
       ['T', '', '']], dtype=object)

In [115]:
columns = [chrom, pos, ref, alt, qual, filter_pass]
records = list(zip(*columns))
dtype = [('CHROM', 'S4'),
         ("POS", "u4"),
         ("REF", "S1"),
         ("ALT", ("S1", 3)),
         ("QUAL", "f4"),
         ("FILTER_PASS", bool)]
vt = allel.VariantTable(records, dtype=dtype)

In [116]:
vt

<VariantTable shape=(2774845,) dtype=(numpy.record, [('CHROM', 'S4'), ('POS', '<u4'), ('REF', 'S1'), ('ALT', 'S1', (3,)), ('QUAL', '<f4'), ('FILTER_PASS', '?')])>
[(b'chrX',   2500022, b'G', [b'A', b'', b''],   4637.71,  True)
 (b'chrX',   2500091, b'T', [b'G', b'', b''],    735.52,  True)
 (b'chrX',   2500194, b'G', [b'T', b'', b''], 107936.72,  True) ...
 (b'chrX', 139999889, b'G', [b'A', b'', b''],    884.66,  True)
 (b'chrX', 139999944, b'G', [b'A', b'', b''],  11669.72,  True)
 (b'chrX', 139999992, b'C', [b'T', b'', b''],  13322.75,  True)]

In [117]:
vt.to_vcf("test.vcf")

In [118]:
test_vcf = allel.read_vcf("test.vcf")

In [119]:
test_vcf

{'variants/ALT': array([['A', '', ''],
        ['G', '', ''],
        ['T', '', ''],
        ...,
        ['A', '', ''],
        ['A', '', ''],
        ['T', '', '']], dtype=object),
 'variants/CHROM': array(['chrX', 'chrX', 'chrX', ..., 'chrX', 'chrX', 'chrX'], dtype=object),
 'variants/FILTER_PASS': array([ True,  True,  True, ...,  True,  True,  True]),
 'variants/ID': array(['.', '.', '.', ..., '.', '.', '.'], dtype=object),
 'variants/POS': array([  2500022,   2500091,   2500194, ..., 139999889, 139999944,
        139999992], dtype=int32),
 'variants/QUAL': array([  4637.71,    735.52, 107936.72, ...,    884.66,  11669.72,
         13322.75], dtype=float32),
 'variants/REF': array(['G', 'T', 'G', ..., 'G', 'G', 'C'], dtype=object)}